# Multiple Linear Regression: Boston House Prediction

* CRIM per capita crime rate by town
* ZN proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS proportion of non-retail business acres per town
* CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* NOX nitric oxides concentration (parts per 10 million)
* RM average number of rooms per dwelling
* AGE proportion of owner-occupied units built prior to 1940
* DIS weighted distances to five Boston employment centres
* RAD index of accessibility to radial highways
* TAX full-value property-tax rate per 10,000usd
* PTRATIO pupil-teacher ratio by town
* B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT % lower status of the population

In [36]:
import pandas as pd

In [37]:
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
         'DIS', 'RAD', 'TAX', 'PTRATIO','B', 'LSTAT', 'MEDV']
df = pd.read_csv('data/housing.data', header=None, delim_whitespace=True, names=names)

In [38]:
df = df.select_dtypes(float)

In [39]:
df.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,222.0,18.7,396.90,5.33,36.2


In [40]:
corr = abs(df.corr()['MEDV']).sort_values(ascending=False)
clmns = corr[1:3].index

In [42]:
df = df.loc[:, list(clmns) + ['MEDV']]
df.head()

,LSTAT,RM,MEDV
0,4.98,6.575,24.0
1,9.14,6.421,21.6
2,4.03,7.185,34.7
3,2.94,6.998,33.4
4,5.33,7.147,36.2


## Create the regressor X matrix and regressand y matrix

In [61]:
X = df.iloc[:, 0:2]
X["Intercept"] = 1
X = X.iloc[:, ::-1]
X = X.values

y = df.iloc[:, -1].values.reshape(-1, 1)

In [69]:
X.shape, y.shape

((506, 3), (506, 1))

In [118]:
n, k = X.shape


## Find the estimate of Beta

In [71]:
import numpy as np
from numpy.linalg import inv

In [82]:
A = inv(X.T@X)

In [85]:
beta_hat = A@X.T@y
beta_hat

array([[-1.35827281],
       [ 5.09478798],
       [-0.64235833]])

## Obtain the estimate of y (predicted y)

In [86]:
y_hat = X@beta_hat

## Compute the residual sum of squares

In [186]:
u_hat = y - y_hat
rss = (u_hat.T@ u_hat).ravel()[0]

## Obtain the variance of beta_hat

In [188]:
beta_hat_var = (rss/(n-k))*(A)

In [189]:
beta_hat_var

array([[ 1.00668361e+01, -1.39248641e+00, -9.91781334e-02],
       [-1.39248641e+00,  1.97549581e-01,  1.19306696e-02],
       [-9.91781334e-02,  1.19306696e-02,  1.91244101e-03]])

## Create the regression table

## Create the table for hypothesis testing

In [190]:
beta_hat_stderr = np.diag(beta_hat_var)**(1/2)

In [191]:
beta_hat.flatten()

array([-1.35827281,  5.09478798, -0.64235833])

In [192]:
[beta_hat.flatten(), beta_hat_stderr]

[array([-1.35827281,  5.09478798, -0.64235833]),
 array([3.17282778, 0.4444655 , 0.04373146])]

In [193]:
df_hyp = pd.DataFrame({'Estimate': beta_hat.flatten(),
                       'Standard_error': beta_hat_stderr})

df_hyp['t_statistic'] = df_hyp['Estimate']/df_hyp['Standard_error']

In [194]:
df_hyp

,Estimate,Standard_error,t_statistic
0,-1.358273,3.172828,-0.428095
1,5.094788,0.444466,11.462730
2,-0.642358,0.043731,-14.688699


In [195]:
from scipy.stats import t as tdist

critical_val = abs(tdist.ppf(q=[0.025], df=n-k))[0]
critical_val

1.9646914053628335

In [196]:
df_hyp['significant_0.05'] = (abs(df_hyp['t_statistic']) > critical_val)
df_hyp

,Estimate,Standard_error,t_statistic,significant_0.05
0,-1.358273,3.172828,-0.428095,False
1,5.094788,0.444466,11.462730,True
2,-0.642358,0.043731,-14.688699,True


## Calculate TSS, ESS and RSS

In [197]:
tss = y.var()*y.size
ess = tss - rss

In [202]:
R_sq = (ess/tss)
R_sq_adj = (1 - (rss/(n-k))/(tss/(n-1)))

print('R squared: {:.4f}'.format(R_sq))
print('R squared (Adjusted): {:.4f}'.format(R_sq_adj))

R squared: 0.6386
R squared (Adjusted): 0.6371


## Make the LinearRegression Model

In [203]:
from sklearn.base import BaseEstimator, ClassifierMixin

In [305]:
class LinearReg(BaseEstimator, ClassifierMixin):
    
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
    
    
    def _remake_X(self, X):
        if self.fit_intercept:
            return np.hstack((np.ones((X.shape[0], 1)), X))
        else:
            return X
    
    @staticmethod
    def _remake_y(y):
        return y.reshape(-1, 1) # Check before reshaping?

    
    def fit(self, X, y):
        
        # Make proper X and y matrix
        X = self._remake_X(X)
        y = self._remake_y(y)
        
        # Store n and k
        n, k = X.shape
        
        # Obtain estimate of beta =  beta_hat
        A = inv(X.T@X)
        beta_hat = A@X.T@y
        
        # Obtain estimate of y = y_hat
        y_hat = X@beta_hat
        
        # Compute residual sum of squares
        u_hat = y - y_hat
        rss = (u_hat.T @ u_hat).ravel()[0]
        
        # Estimate the variance of beta_hat
        beta_hat_var = (rss/(n-k))*A
        
        # Calculate tss, ess and rss
        tss = y.var()*n
        ess = tss - rss
        
        # Calculate R-squared and Adjusted R-squared
        rsq = ess/tss
        rsq_adj = 1 - (rss/(n-k))/(tss/(n-1))
        
        # store
        self.n = n
        self.k = k
        self.beta_hat = beta_hat
        self.rss = rss
        self.beta_hat_var = beta_hat_var
        self.u_hat = u_hat
        self.rsq = rsq
        self.rsq_adj = rsq_adj
        
        return self
    
    def predict(self, X):
        X = self._remake_X(X)
        return X@self.beta_hat
    
    def score(self, X, y):
        lm = LinearReg(self.fit_intercept)
        lm.fit(X, y)
        return lm.rsq_adj
    

In [306]:
X = df.iloc[:, 0:2]
X = X.iloc[:, ::-1]
X = X.values

y = df.iloc[:, -1].values.reshape(-1, 1)

In [307]:
lm = LinearReg()
lm.fit(X, y)

LinearReg(fit_intercept=True)

In [308]:
lm.score(X, y)

0.6371244754701231

In [309]:
from sklearn.model_selection import cross_val_score

In [312]:
-cross_val_score(lm, X, y, scoring="neg_mean_squared_error")

array([11.78879381, 28.97215949, 47.84395882, 71.77053822, 36.6099291 ])

In [313]:
from sklearn.linear_model import LinearRegression
lm_sk = LinearRegression()

-cross_val_score(lm_sk, X, y, scoring="neg_mean_squared_error")

array([11.78879381, 28.97215949, 47.84395882, 71.77053822, 36.6099291 ])

In [314]:
cross_val_score(lm_sk, X, y, cv=10) # Which default scoring metric does this use?

array([ 0.66158301,  0.64905012, -1.51946963,  0.49331255,  0.59340312,
        0.47386502, -0.17695506,  0.05396455, -1.77172578,  0.21261084])

## Joint Hypothesis

$ H_{0}: \beta_{2} +\beta_{3} = 0$ <br/>
$ H_{1}: \beta_{2} +\beta_{3} \neq 0$

In [351]:
lm = LinearReg()

In [352]:
# Unrestricted residual sum of squares
urss = lm.fit(X, y).rss

In [353]:
X.shape

(506, 2)

In [354]:
X_star = (X[:, 0] - X[:, 1]).reshape(-1, 1)
y_star = y - X[:, 1].reshape(-1, 1)

In [355]:
# Restricted residual sum of squares
rrss = lm.fit(X_star, y_star).rss

num_restr = X_star.shape[1]
n = X.shape[0]
k = X.shape[1]
F_stat = ((rrss - urss)/num_restr)/(urss/(n-k))

print('F statistic: {:.5f}'.format(F_stat))
print('Critical value: {:.5f}'.format(fdistr.ppf(0.05, num_restr, n-k)))

F statistic: 53.47836
Critical value: 0.00394


Now let's do something that might make us not reject the null hypothesis.

In [360]:
lm.fit(X, y)
beta_hat = lm.beta_hat
beta_hat[1] + beta_hat[2]

array([4.45242965])

$ H_{0}: \beta_{2} +\beta_{3} = 4.47$ <br/>
$ H_{1}: \beta_{2} +\beta_{3} \neq 4.47$

In [363]:
X_star = (X[:, 0] - X[:, 1]).reshape(-1, 1)
y_star = y - 4.47*X[:, 1].reshape(-1, 1)

In [364]:
# Restricted residual sum of squares
rrss = lm.fit(X_star, y_star).rss

num_restr = X_star.shape[1]
n = X.shape[0]
k = X.shape[1]
F_stat = ((rrss - urss)/num_restr)/(urss/(n-k))

print('F statistic: {:.5f}'.format(F_stat))
print('Critical value: {:.5f}'.format(fdistr.ppf(0.05, num_restr, n-k)))

F statistic: 0.00139
Critical value: 0.00394
